In [2]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('tripadvisor_hotel_reviews.csv')

In [18]:
df

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5
...,...,...
20486,"best kept secret 3rd time staying charm, not 5...",5
20487,great location price view hotel great quick pl...,4
20488,"ok just looks nice modern outside, desk staff ...",2
20489,hotel theft ruined vacation hotel opened sept ...,1


In [3]:
df.dtypes

Review    object
Rating     int64
dtype: object

<h2>Cleaning Data</h2>

<h3>Cleaning Text Data</h3>

In [6]:
!pip install nltk

In [4]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

In [8]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...


True

In [5]:
def clean_text(text):
    # Tokenize the text
    tokens = word_tokenize(text)

    # Convert tokens to lowercase
    tokens = [token.lower() for token in tokens]

    # Remove punctuation
    tokens = [token for token in tokens if token not in string.punctuation]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join tokens back into a single string
    cleaned_text = ' '.join(tokens)

    return cleaned_text

In [13]:
nltk.download("omw-1.4")

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...


True

In [6]:
df['cleaned_review'] = df['Review'].apply(clean_text)

In [7]:
df

,Review,Rating,cleaned_review
0,nice hotel expensive parking got good deal sta...,4,nice hotel expensive parking got good deal sta...
1,ok nothing special charge diamond member hilto...,2,ok nothing special charge diamond member hilto...
2,nice rooms not 4* experience hotel monaco seat...,3,nice room 4 experience hotel monaco seattle go...
3,"unique, great stay, wonderful time hotel monac...",5,unique great stay wonderful time hotel monaco ...
4,"great stay great stay, went seahawk game aweso...",5,great stay great stay went seahawk game awesom...
...,...,...,...
20486,"best kept secret 3rd time staying charm, not 5...",5,best kept secret 3rd time staying charm 5-star...
20487,great location price view hotel great quick pl...,4,great location price view hotel great quick pl...
20488,"ok just looks nice modern outside, desk staff ...",2,ok look nice modern outside desk staff n't par...
20489,hotel theft ruined vacation hotel opened sept ...,1,hotel theft ruined vacation hotel opened sept ...


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [9]:
vectorizer = CountVectorizer()
dtm = vectorizer.fit_transform(df['cleaned_review'])

In [24]:
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(dtm)

LatentDirichletAllocation(n_components=5, random_state=42)

In [28]:
!pip install gensim

In [15]:
num_components=5

print(f"Number of Components: {num_components}")

# Apply LDA to the document-term matrix
lda = LatentDirichletAllocation(n_components=num_components, random_state=42)
lda.fit(dtm)

# Convert dcument-term matrix to Gensim corpus
corpus = gensim.matutils.Sparse2Corpus(dtm.T)

# Create a dictionary mapping for the corpus
id2word = dict((v, k) for k, v in vectorizer.vocabulary_.items())

# Fit the LDA model using Gensim
lda_gensim = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_components, random_state=42)

Number of Components: 5


In [20]:
df['tokens'].tolist()

[['nice',
  'hotel',
  'expensive',
  'parking',
  'got',
  'good',
  'deal',
  'stay',
  'hotel',
  'anniversary',
  'arrived',
  'late',
  'evening',
  'took',
  'advice',
  'previous',
  'review',
  'valet',
  'parking',
  'check',
  'quick',
  'easy',
  'little',
  'disappointed',
  'non-existent',
  'view',
  'room',
  'room',
  'clean',
  'nice',
  'size',
  'bed',
  'comfortable',
  'woke',
  'stiff',
  'neck',
  'high',
  'pillow',
  'soundproof',
  'like',
  'heard',
  'music',
  'room',
  'night',
  'morning',
  'loud',
  'bang',
  'door',
  'opening',
  'closing',
  'hear',
  'people',
  'talking',
  'hallway',
  'maybe',
  'noisy',
  'neighbor',
  'aveda',
  'bath',
  'product',
  'nice',
  'goldfish',
  'stay',
  'nice',
  'touch',
  'taken',
  'advantage',
  'staying',
  'longer',
  'location',
  'great',
  'walking',
  'distance',
  'shopping',
  'overall',
  'nice',
  'experience',
  'pay',
  '40',
  'parking',
  'night'],
 ['ok',
  'nothing',
  'special',
  'charge',
 

In [22]:
# Compute coherence score for the model
coherence_model = CoherenceModel(model=lda_gensim, texts=df['tokens'], dictionary=Dictionary(df['tokens'].tolist()), coherence='c_v')
coherence_score = coherence_model.get_coherence()
print(f"Coherence Score: {coherence_score}")

Coherence Score: 0.6250585501121998


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
import gensim
from nltk.tokenize import word_tokenize

# Tokenize the cleaned text
df['tokens'] = df['cleaned_review'].apply(word_tokenize)

# Create a dictionary mapping for the corpus
dictionary = Dictionary(df['tokens'])

min_components = 5
max_components = 20

coherance_collect=[]

# Iterate over the number of components
for num_components in range(min_components, max_components+1):
    print(f"Number of Components: {num_components}")
    
    # Apply LDA to the document-term matrix
    lda = LatentDirichletAllocation(n_components=num_components, random_state=42)
    lda.fit(dtm)

    # Convert document-term matrix to Gensim corpus
    corpus = gensim.matutils.Sparse2Corpus(dtm.T)

    # Create a dictionary mapping for the corpus
    id2word = dict((v, k) for k, v in vectorizer.vocabulary_.items())

    # Fit the LDA model using Gensim
    lda_gensim = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_components, random_state=42)

    # Compute coherence score for the model
    coherence_model = CoherenceModel(model=lda_gensim, texts=df['tokens'], dictionary=Dictionary(df['tokens'].tolist()), coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print(f"Coherence Score: {coherence_score}")
    coherance_collect.append(coherence_score)

#     # Print the top words for each topic
#     topics = lda_gensim.show_topics(num_topics=num_components, num_words=10)
#     for topic in topics:
#         print(f"Topic #{topic[0] + 1}: {topic[1]}")

#     # Extract topic weights for each document
#     topic_weights = np.zeros((len(df), num_components))
#     for i, doc in enumerate(corpus):
#         topic_dist = lda_gensim.get_document_topics(doc)
#         for topic_id, topic_prob in topic_dist:
#             topic_weights[i, topic_id] = topic_prob

#     fig, ax = plt.subplots(figsize=(10, 6))
#     ax.bar(topic_weights[:,0].astype(int), topic_weights[:,1].astype(float), width=0.5, alpha=0.7)
#     ax.set_xlabel('Topic')
#     ax.set_ylabel('Weight')
#     ax.set_title(f'Topic Weights for {num_components} Topics')
#     plt.show()

Number of Components: 5


In [14]:
df['tokens']

0        [nice, hotel, expensive, parking, got, good, d...
1        [ok, nothing, special, charge, diamond, member...
2        [nice, room, 4, experience, hotel, monaco, sea...
3        [unique, great, stay, wonderful, time, hotel, ...
4        [great, stay, great, stay, went, seahawk, game...
                               ...                        
20486    [best, kept, secret, 3rd, time, staying, charm...
20487    [great, location, price, view, hotel, great, q...
20488    [ok, look, nice, modern, outside, desk, staff,...
20489    [hotel, theft, ruined, vacation, hotel, opened...
20490    [people, talking, ca, n't, believe, excellent,...
Name: tokens, Length: 20491, dtype: object

In [26]:
# Visualize the output using a bar plot
topic_weights = lda_gensim.show_topics(formatted=False)
#topic_weights = np.array(topic_weights)
topic_weights

[(0,
  [('8am-11am', 0.047986504),
   ('stayed.-', 0.027198734),
   ('front/lobby', 0.016282592),
   ('terriffic', 0.011339087),
   ('package.this', 0.010995166),
   ('eyeglass', 0.010790378),
   ('mentality', 0.010745108),
   ('staff.attentive', 0.009048005),
   ('affect', 0.008554898),
   ('contributor', 0.008444051)]),
 (1,
  [('willie', 0.0140391495),
   ('mannnnn', 0.01240447),
   ('fajitas', 0.010968294),
   ('stayed.-', 0.010937294),
   ('emphasized', 0.010760255),
   ('9.30pm', 0.010088285),
   ('sacred', 0.010048261),
   ('front/lobby', 0.009387815),
   ('eyeglass', 0.009036661),
   ('hardenberg', 0.0068499874)]),
 (2,
  [('8am-11am', 0.011201477),
   ('2ish', 0.006387031),
   ('stayed.-', 0.0051479824),
   ('served.the', 0.005135904),
   ('9.30pm', 0.0047301333),
   ('allergo', 0.0044662203),
   ('front/lobby', 0.004324072),
   ('days.with', 0.0043234997),
   ('terriffic', 0.004120182),
   ('emphasized', 0.004017721)]),
 (3,
  [('stayed.-', 0.037963204),
   ('8am-11am', 0.020

In [30]:
# Extract topic weights for each document
topic_weights = np.zeros((len(df), num_components))
for i, doc in enumerate(corpus):
    topic_dist = lda_gensim.get_document_topics(doc)
    for topic_id, topic_prob in topic_dist:
        topic_weights[i, topic_id] = topic_prob

In [31]:
topic_weights

array([[0.64515579, 0.        , 0.        , 0.34748998, 0.        ],
       [0.26126465, 0.        , 0.        , 0.73611057, 0.        ],
       [0.21162671, 0.        , 0.        , 0.78533965, 0.        ],
       ...,
       [0.3006869 , 0.        , 0.        , 0.68962449, 0.        ],
       [0.14628458, 0.0154856 , 0.        , 0.83768463, 0.        ],
       [0.31394097, 0.        , 0.        , 0.45671996, 0.22446875]])